In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

kuka_output = pd.read_excel('KukaOutput-ICNAP.xlsx', engine='openpyxl')
kuka_output['EndDate'] = pd.to_datetime(kuka_output['EndDate'])
t0 = kuka_output['EndDate'].iloc[0]
kuka_output['EndDate min'] = (kuka_output['EndDate'] - t0).astype('timedelta64[m]')
programs = pd.read_excel('OvenPrograms-ICNAP.xlsx', engine='openpyxl')

In [37]:
class Oven:
    def __init__(self, index, finish_time=0, temp=0, rotor_list=[]):
        self.index = index
        self.finish_time = finish_time
        self.temp = temp
        self.rotor_list = rotor_list
    
    def get_time_per_product(self, time_available, duration, l):
        if time_available > self.finish_time:
            return time_available + (duration/l)
        return self.finish_time + (duration/l)

    def get_enter_time(self, time_available):
        if time_available > self.finish_time:
            return time_available
        return self.finish_time
    
    def dif_temperature(self, target_temp):
        return target_temp-self.temp
    
    def add_rotor(self, time_available, duration, cold_temp, index):
        self.temp = cold_temp
        self.rotor_list.append(index)
        if time_available > self.finish_time:
            self.finish_time = time_available + duration
            
        else:
            self.finish_time += duration
        
class Node:
    def __init__(self, path, remaining_df, ovens, node_dict):
        self.path = path
        self.remaining_df = remaining_df
        self.ovens = ovens
        self.node_dict = node_dict
        
    def get_neighboors(self):
        if self.remaining_df.shape[0] > 10:
            max_iter = 8
        else:
            max_iter = self.remaining_df.shape[0]
        
        ns = [self.get_neighboor(i) for i in range(max_iter)]
        return list(filter(lambda n: n['Lambda']<=max_lambda, ns))
            
    def get_next(self):
        neighboors = self.get_neighboors()
        best_cost = min([n['MinCost'] for n in neighboors])
        next_n_index = [n['MinCost'] for n in neighboors].index(min([n['MinCost'] for n in neighboors]))
        return neighboors[next_n_index]
    
    def get_neighboor(self, index):
        n = dict()
        n['Product'] = self.remaining_df['Product'].iloc[index]
        n['TimeAvailable'] = self.remaining_df['EndDate min'].iloc[index]
        n['Duration'] = programs[programs['product type']==n['Product']]['program duration (min)'].values[0] + 60
        n['TargetTemp'] = programs[programs['product type']==n['Product']]['target temp (C)'].values[0]
        n['CoolTemp'] = programs[programs['product type']==n['Product']]['cooling temp  (C)'].values[0]
        df_index = self.remaining_df.iloc[:index+1]
        n['Lambda'] = df_index[df_index['Product']==n['Product']].shape[0]
        n['ProductIndexes'] = list(df_index[df_index['Product']==n['Product']].index)
        n['ProductTimes'] = [oven.get_time_per_product(n['TimeAvailable'], n['Duration'], n['Lambda']) for oven in self.ovens]
        n['ProductDifTemps'] = [oven.dif_temperature(n['TargetTemp'])/n['Lambda'] for oven in self.ovens]
        n['Costs'] = [self.calculate_cost(n['ProductTimes'][i], n['ProductDifTemps'][i]) for i in range(len(n['ProductTimes']))]
        n['MinCost'] = min(n['Costs'])
        n['OvenIndex'] = n['Costs'].index(n['MinCost'])
        n['EnterTime'] = self.ovens[n['OvenIndex']].get_enter_time(n['TimeAvailable'])
        n['DifTemperature'] = self.ovens[n['OvenIndex']].dif_temperature(n['TargetTemp'])
        n['TimesAvailables'] = self.remaining_df['EndDate min'].loc[n['ProductIndexes']]
        n['WaitingTimes'] = [n['EnterTime']-time for time in n['TimesAvailables']]
        n['SumWaitingTime'] = sum(n['WaitingTimes'])
        n['MeanWaitingTime'] = sum(n['WaitingTimes'])/len(n['WaitingTimes'])
        return n
    
    def calculate_cost(self, time, temp):
        percentage_temp = temp*0.3/291
        percentage_time = time*0.7/920
        return percentage_temp+percentage_time

In [38]:
oven_list = [Oven(i, finish_time=0, temp=0, rotor_list=[]) for i in range(6)]
path = []
df_remaining = kuka_output.copy()
current_node = Node(path, df_remaining, oven_list, {})
max_lambda = 3

while df_remaining.shape[0]>0:
    next_node_dict = current_node.get_next()
    oven_list[next_node_dict['OvenIndex']].add_rotor(next_node_dict['TimeAvailable'], next_node_dict['Duration'],
                                                     next_node_dict['CoolTemp'], next_node_dict['ProductIndexes'])
    df_remaining.drop(index=next_node_dict['ProductIndexes'], inplace=True)
    path.append(current_node)
    current_node = Node(path, df_remaining, oven_list, next_node_dict)
    if df_remaining.shape[0] == 0:
        path.append(current_node)

node_dicts = [i.node_dict for i in path][1:]
nodes_df = pd.DataFrame(node_dicts)
nodes_df['JobStart'] = t0 + pd.to_timedelta(nodes_df['EnterTime'], unit='m')
nodes_df['JobFinish'] = t0 + pd.to_timedelta(nodes_df[['EnterTime', 'Duration']].sum(axis=1), unit='m')
nodes_df['ProductIndexesText'] = nodes_df['ProductIndexes'].astype(str)
nodes_df.head()

,Product,TimeAvailable,Duration,TargetTemp,CoolTemp,Lambda,ProductIndexes,ProductTimes,ProductDifTemps,Costs,...,OvenIndex,EnterTime,DifTemperature,TimesAvailables,WaitingTimes,SumWaitingTime,MeanWaitingTime,JobStart,JobFinish,ProductIndexesText
0,Rotor A3,1.0,450,319.55085,42.60678,2,"[0, 1]","[226.0, 226.0, 226.0, 226.0, 226.0, 226.0]","[159.775425, 159.775425, 159.775425, 159.77542...","[0.33667345472882115, 0.33667345472882115, 0.3...",...,0,1.0,319.55085,"0 0.0 1 1.0 Name: EndDate min, dtype: fl...","[1.0, 0.0]",1.0,0.500000,2022-02-01 01:02:47,2022-02-01 08:32:47,"[0, 1]"
1,Rotor A3,338.0,450,319.55085,42.60678,2,"[4, 5]","[676.0, 563.0, 563.0, 563.0, 563.0, 563.0]","[138.472035, 159.775425, 159.775425, 159.77542...","[0.6571025013446885, 0.593086498207082, 0.5930...",...,1,338.0,319.55085,"4 199.0 5 338.0 Name: EndDate min, dtype...","[139.0, 0.0]",139.0,69.500000,2022-02-01 06:39:47,2022-02-01 14:09:47,"[4, 5]"
2,Rotor B3,532.0,300,326.65198,71.01130,3,"[8, 9, 10]","[632.0, 888.0, 632.0, 632.0, 632.0, 632.0]","[94.68173333333333, 94.68173333333333, 108.883...","[0.5784795995816524, 0.7732622082773045, 0.593...",...,0,532.0,284.04520,8 525.0 9 532.0 10 532.0 Name: EndD...,"[7.0, 0.0, 0.0]",7.0,2.333333,2022-02-01 09:53:47,2022-02-01 14:53:47,"[8, 9, 10]"
3,Rotor A2,365.0,450,319.55085,42.60678,2,"[3, 7]","[1057.0, 1013.0, 590.0, 590.0, 590.0, 590.0]","[124.26977500000001, 138.472035, 159.775425, 1...","[0.9323523005378753, 0.9135155448229493, 0.613...",...,2,365.0,319.55085,"3 136.0 7 365.0 Name: EndDate min, dtype...","[229.0, 0.0]",229.0,114.500000,2022-02-01 07:06:47,2022-02-01 14:36:47,"[3, 7]"
4,Rotor A4,127.0,460,319.55085,42.60678,1,[2],"[1292.0, 1248.0, 1275.0, 587.0, 587.0, 587.0]","[248.53955000000002, 276.94407, 276.94407, 319...","[1.2392698184670552, 1.2350745679067683, 1.255...",...,3,127.0,319.55085,"2 127.0 Name: EndDate min, dtype: float64",[0.0],0.0,0.000000,2022-02-01 03:08:47,2022-02-01 10:48:47,[2]


In [39]:
fig = px.timeline(nodes_df, x_start='JobStart', x_end='JobFinish', y="OvenIndex", color="Product", title="Gantt Chart")
fig.update_yaxes(autorange="reversed")
fig.show()

In [18]:
nodes_df['DifTemperature'].sum()

79482.94809000002

In [ ]:
nodes_df['DifTemperature'].sum()

In [27]:
oven_6_time = nodes_df[nodes_df['OvenIndex']==5]['Duration'].sum()

In [28]:
total_process_time = nodes_df[['EnterTime', 'Duration']].sum(axis=1).max()

In [29]:
oven_6_time/total_process_time

0.0997688283246137

In [40]:
px.histogram(nodes_df, x='MeanWaitingTime')

In [45]:
waiting_list_times = []
for node in nodes_df['WaitingTimes']:
    for n in node:
        waiting_list_times.append(n)

waiting_list_times

[1.0,
 0.0,
 139.0,
 0.0,
 7.0,
 0.0,
 0.0,
 229.0,
 0.0,
 0.0,
 0.0,
 173.0,
 92.0,
 0.0,
 0.0,
 0.0,
 342.0,
 144.0,
 0.0,
 3.0,
 2.0,
 0.0,
 2.0,
 0.0,
 121.0,
 0.0,
 118.0,
 0.0,
 234.0,
 34.0,
 0.0,
 86.0,
 0.0,
 100.0,
 98.0,
 0.0,
 417.0,
 137.0,
 0.0,
 64.0,
 62.0,
 0.0,
 82.0,
 0.0,
 35.0,
 34.0,
 0.0,
 202.0,
 2.0,
 0.0,
 0.0,
 8.0,
 0.0,
 15.0,
 0.0,
 0.0,
 96.0,
 0.0,
 404.0,
 101.0,
 0.0,
 84.0,
 0.0,
 0.0,
 133.0,
 0.0,
 11.0,
 0.0,
 40.0,
 0.0,
 224.0,
 83.0,
 0.0,
 0.0,
 107.0,
 0.0,
 0.0,
 0.0,
 23.0,
 1.0,
 0.0,
 107.0,
 0.0,
 0.0,
 0.0,
 270.0,
 0.0,
 160.0,
 0.0,
 211.0,
 0.0,
 296.0,
 133.0,
 0.0,
 163.0,
 0.0,
 148.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 319.0,
 136.0,
 0.0,
 1.0,
 0.0,
 0.0,
 205.0,
 0.0,
 163.0,
 0.0,
 195.0,
 0.0,
 80.0,
 0.0,
 142.0,
 0.0,
 153.0,
 0.0,
 219.0,
 3.0,
 0.0,
 235.0,
 0.0,
 1.0,
 0.0,
 157.0,
 0.0,
 0.0,
 207.0,
 0.0,
 22.0,
 0.0,
 259.0,
 113.0,
 0.0,
 0.0,
 155.0,
 0.0,
 0.0,
 0.0,
 204.0,
 0.0,
 148.0,
 0.0,
 0.0,
 237.0,
 90.

In [50]:
fig = px.histogram(x=waiting_list_times, title="Idle Histogram with stack")
fig.update_layout(yaxis_range=[0,300],xaxis_range=[0,4500])
fig.show()

In [52]:
sum(waiting_list_times)/len(waiting_list_times)

66.41619585687383